# Reasoning Model


## Step 0.

In [11]:
!pip install sacrebleu


  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/cd/45/7b55a7bd7e5c5b573b40ad58ba43fa09962dc5c8d71b1f573d4aeaa54a7e/sacrebleu-2.5.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/51.8 kB ? eta -:--:--
     ---------------------------------------- 51.8/51.8 kB 2.8 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/f7/60/1974cfdd5bb770568ddc6f89f3e0df4cfdd1acffd5a609dff5e95f48c6e2/portalocker-3.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/104.1 kB ? eta -:--:--
   ---------------------------------------- 104.1/104.1 kB 5.9 MB/s eta 0:00:00


In [1]:
# ------------------------------
# ✅ 0. Install Dependencies
# ------------------------------
!pip install torch transformers --upgrade

In [2]:
# ------------------------------
# ✅ 1. Imports and Settings
# ------------------------------
import os
from transformers import pipeline

In [5]:
# ------------------------------
# ✅ 2. Load Pretrained Reasoning Model
# ------------------------------
# Fast, instruction-tuned model — no training needed
model_name = "MBZUAI/LaMini-Flan-T5-783M"
reasoning_pipeline = pipeline("text2text-generation", model=model_name)

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

c:\Users\anani\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anani\.cache\huggingface\hub\models--MBZUAI--LaMini-Flan-T5-783M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [6]:
# ------------------------------
# ✅ 3. Reasoning from Healthcare Claims
# ------------------------------
# Each tuple: (claim text, credibility score)
claims = [
    ("Vaccines cause autism.", 5),
    ("Exercise reduces the risk of heart disease.", 90),
    ("Drinking 8 glasses of water is essential daily.", 40),
    ("Eating sugar causes diabetes.", 30),
    ("Probiotics help with IBS symptoms.", 70)
]

# Generate and print explanations
for claim, score in claims:
    reasoning_type = "accurate" if score > 70 else "misinformation"
    prompt = f"Claim: {claim}\nCredibility: {score}%\nExplain why this claim is likely {reasoning_type}."
    output = reasoning_pipeline(prompt, max_length=128, num_return_sequences=1)[0]["generated_text"]
    print(f"\n📝 Claim: {claim}\n🔢 Score: {score}%\n💬 Explanation: {output}")



📝 Claim: Vaccines cause autism.
🔢 Score: 5%
💬 Explanation: This claim is likely misinformation because there is no scientific evidence to support the claim that vaccines cause autism. Numerous studies have shown that vaccines do not cause autism, and there is no scientific evidence to support this claim. Additionally, vaccines are safe and effective in preventing diseases, and there is no scientific evidence to suggest that autism is caused by vaccines. Therefore, this claim is not supported by scientific evidence and is not supported by any credible sources.

📝 Claim: Exercise reduces the risk of heart disease.
🔢 Score: 90%
💬 Explanation: The claim is likely accurate because regular exercise has been shown to lower the risk of heart disease by up to 90%. This is because exercise helps to strengthen the heart muscle, which is responsible for pumping blood throughout the body. Additionally, exercise can also lower blood pressure, cholesterol, and blood sugar levels, which are all risk 

In [7]:
# ------------------------------
# ✅ 4. Save Model + Tokenizer for Streamlit
# ------------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

save_path = "flan_reasoning_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("\nModel and tokenizer saved to 'flan_reasoning_model'")


Model and tokenizer saved to 'flan_reasoning_model'


# Reasoning Model TEST

In [1]:
# ----------------------------------------
# 📘 1. Install Dependencies
# ----------------------------------------
!pip install transformers datasets evaluate --upgrade

In [2]:
# ----------------------------------------
# 📘 2. Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
import re
import evaluate
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# ----------------------------------------
# 📘 3. Load and Clean Dataset
# ----------------------------------------
df = pd.read_csv("claim_explanations.csv")
df = df.dropna(subset=["statement", "target"])

# Generate synthetic credibility score between 10–90%
df["credibility"] = np.random.randint(10, 91, df.shape[0])

# Strip HTML tags from explanations
def strip_html(text):
    return re.sub(r'<[^>]+>', '', str(text))

df["expected_explanation"] = df["target"].apply(strip_html)
df["input_text"] = df.apply(lambda row: f"Claim: {row['statement']} Credibility: {row['credibility']}%", axis=1)
df = df.reset_index(drop=True)


In [4]:
# ----------------------------------------
# 📘 4. Load LaMini-Flan-T5-783M Model
# ----------------------------------------
tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Flan-T5-783M")
model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-Flan-T5-783M")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [5]:
# ----------------------------------------
# 📘 5. Generate Model Explanations (Optimized with Batch + Progress)
# ----------------------------------------
tqdm.pandas()

# Split input prompts into batches
batch_size = 8  # adjust based on your memory/GPU
prompts = df["input_text"].tolist()
generated_outputs = []

print("⏳ Generating explanations in batches...")
for i in tqdm(range(0, len(prompts), batch_size)):
    batch = prompts[i:i + batch_size]
    batch_results = pipe(batch, max_length=100, do_sample=False)
    generated_outputs.extend([res["generated_text"] for res in batch_results])

# Assign to DataFrame
df["generated_explanation"] = generated_outputs


⏳ Generating explanations in batches...


100%|██████████| 326/326 [1:25:11<00:00, 15.68s/it]


In [13]:
output_path = "lamini_generated_explanations.csv"
df.to_csv(output_path, index=False)

print(f"Explanations saved to: {output_path}")

Explanations saved to: lamini_generated_explanations.csv


In [12]:
import re
import evaluate

# ----------------------------------------
# 📘 Step 1: Clean and Truncate Long Texts
# ----------------------------------------
def clean_text(text):
    text = str(text)
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text[:300]  # optional truncation

df_clean = df.dropna(subset=["generated_explanation", "expected_explanation"]).copy()
df_clean["generated_explanation"] = df_clean["generated_explanation"].apply(clean_text)
df_clean["expected_explanation"] = df_clean["expected_explanation"].apply(clean_text)

# Filter out blank rows (empty strings)
df_clean = df_clean[
    (df_clean["generated_explanation"].str.strip() != "") &
    (df_clean["expected_explanation"].str.strip() != "")
]

preds = df_clean["generated_explanation"].tolist()
refs = df_clean["expected_explanation"].tolist()

# ----------------------------------------
# 📘 Step 2: Evaluate ROUGE (String Matching)
# ----------------------------------------
rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions=preds, references=refs)

print("\nROUGE Results:")
for k, v in rouge_results.items():
    print(f"{k}: {v:.4f}")

# ----------------------------------------
# 📘 Step 3: Evaluate BLEU using sacrebleu (Robust)
# ----------------------------------------
import sacrebleu

# BLEU expects list of string predictions and list of list of references
bleu_score = sacrebleu.corpus_bleu(preds, [refs])  # refs must be list of lists

print("\nBLEU Score (sacrebleu):")
print(f"BLEU: {bleu_score.score:.2f}")




ROUGE Results:
rouge1: 0.0815
rouge2: 0.0192
rougeL: 0.0610
rougeLsum: 0.0608

BLEU Score (sacrebleu):
BLEU: 0.47


In [14]:
# ----------------------------------------
# 📘 8. Save Model and Tokenizer for Streamlit
# ----------------------------------------
model.save_pretrained("lamini_flant5_model")
tokenizer.save_pretrained("lamini_flant5_model")
print("\nModel and tokenizer saved to 'lamini_flant5_model'")

print("\nEvaluation complete and explanations generated.")


Model and tokenizer saved to 'lamini_flant5_model'

Evaluation complete and explanations generated.


In [ ]:
# ----------------------------------------
# 🧪 Test Reasoning Model Interactively
# ----------------------------------------

from transformers import pipeline

# Load from HuggingFace or your local saved model
# Use "lamini_flant5_model" if you've already saved it earlier
reasoning_pipe = pipeline(
    "text2text-generation",
    model="MBZUAI/LaMini-Flan-T5-783M",  # or "lamini_flant5_model" if local
    tokenizer="MBZUAI/LaMini-Flan-T5-783M"
)

# Interactive test
while True:
    claim = input("Enter a healthcare claim (or 'exit' to stop): ")
    if claim.lower() == "exit":
        break

    credibility = input("Enter a credibility % (0-100): ")
    try:
        score = int(credibility)
        prompt = f"Claim: {claim} Credibility: {score}%"
        output = reasoning_pipe(prompt, max_length=100, do_sample=False)[0]["generated_text"]
        print(f"\n🧠 Explanation: {output}\n")
    except ValueError:
        print("Please enter a valid number for credibility.")
